Coisas pra fazer:
- Voltar no tratamento de outliers e mudar a forma (estou fazendo com média)
- Voltar na parte que excluo colunas com suporte muito baixo, talvez colocar esses casos para uma coluna "outro"
- Transformar as colunas não numéricas em numéricas
- Ver como selecionar as dimensões
- Ver quais métodos seriam mais adequados
- Como medir os resultados
-

In [95]:
import pandas as pd
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN, dbscan, KMeans
import numpy as np
from utils import *

In [96]:
df = pd.read_csv('preprocessing_result1.csv')
df.drop(columns=['likes','num_comments'], inplace=True)
df

,content_type,tone,emotion_displayed,location_type,followers,has_a_person,vape_present,is_vaping,img_mentions_brands,text_mentions_brands,...,post_style_array_Informational,post_style_array_Promotion,post_style_array_Announcement,type_of_content_array_Initiative,type_of_content_array_Advertisement,type_of_content_array_Promotion,type_of_content_array_Health,type_of_content_array_Campaign,type_of_content_array_Informational,type_of_content_array_Warning
0,Image,Negative,other,Public,0.668284,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Image,Negative,other,Public,0.058955,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Image,Negative,other,Indoor,0.891045,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Image,Sarcastic,annoyance,Indoor,0.002985,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Image,Positive,other,Private,0.118284,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,Image,Negative,other,Public,0.067910,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
979,Image,Positive,happiness,Public,0.972761,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
980,Image,Negative,other,Public,0.003731,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
981,Image,Positive,happiness,Outdoor,0.194030,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
def binarize_categorical_columns(df, columns):
    for column in columns:
        dummies = pd.get_dummies(df[column], prefix=column)
        df = pd.concat([df, dummies], axis=1)
        df.drop(column, axis=1, inplace=True)
    return df

In [98]:
df = binarize_categorical_columns(df, ['content_type', 'tone', 'emotion_displayed', 'location_type'])
#df.drop(columns=['content_type', 'tone', 'emotion_displayed', 'location_type'], inplace=True)

In [99]:
df.dtypes

followers                float64
has_a_person             float64
vape_present             float64
is_vaping                float64
img_mentions_brands      float64
                          ...   
location_type_Indoor        bool
location_type_Neutral       bool
location_type_Outdoor       bool
location_type_Private       bool
location_type_Public        bool
Length: 106, dtype: object

In [100]:
df.columns.tolist()


['followers',
 'has_a_person',
 'vape_present',
 'is_vaping',
 'img_mentions_brands',
 'text_mentions_brands',
 'Health_ad',
 'are_there_children',
 'sensitive_content_bool',
 'video_link_count',
 'people_gender_Female',
 'people_gender_Other',
 'people_gender_Male',
 'people_skin_color_dark',
 'people_skin_color_light',
 'people_skin_color_medium',
 'people_age_group_Young',
 'people_age_group_Teen',
 'people_age_group_Old',
 'people_age_group_Adult',
 'people_physical_attributes_muscular',
 'people_physical_attributes_average',
 'people_common_appearances_sporty',
 'people_common_appearances_professional',
 'people_common_appearances_casual',
 'people_common_appearances_formal',
 'people_hair_color_brown',
 'people_hair_color_grey',
 'people_hair_color_blonde',
 'people_hair_color_red',
 'people_hair_color_white',
 'people_hair_color_other',
 'people_hair_color_black',
 'target_demographic_Black community',
 'target_demographic_adults',
 'target_demographic_teenagers',
 'target_demog

In [101]:
# # make all columns booleans
# for column in df.columns.tolist()[1:]:
#     if df[column].dtype == 'float64':
#         df[column] = df[column].astype('bool')
#df

In [102]:
# reduced_df = df[['followers', 'has_a_person', 'vape_present']]
# reduced_df.plot(kind='scatter', x='followers', y='has_a_person', c='vape_present', colormap='viridis')

In [103]:
empirical_n_clusters = np.sqrt(len(df))/2
int(empirical_n_clusters)

15

In [118]:
dbscan_clustering = DBSCAN(eps=3, min_samples=2).fit(df)

silhouette_score(df,dbscan_clustering.labels_)

0.22962218546170737

In [105]:
clustering

,eps,3
,min_samples,2
,metric,'euclidean'
,metric_params,None
,algorithm,'auto'
,leaf_size,30
,p,None
,n_jobs,None


In [106]:
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(df)
#kmeans.labels_
silhouette_score(df,kmeans.labels_)

0.08204026146736215